<a href="https://colab.research.google.com/github/Prathambiradr12345/Natural-Langauge-Proccessing/blob/main/bow_with_advanced_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Token Features

cwc_min: This is the ratio of the number of common words to the length of the smaller question

eg:#common words/min(words(q1,q2))(jo dono me se bada (q1,q2) lenge)



cwc_max: This is the ratio of the number of common words to the length of the larger question

eg:#common words/max(words(q1,q2))(jo dono me se bada (q1,q2) lenge)


csc_min: This is the ratio of the number of common stop words to the smaller stop word count among the two questions

Eg:common words stopwords/min(stop words(q1,q2)
)
csc_max: This is the ratio of the number of common stop words to the larger stop word count among the two questions

Eg:#common words stopwords/max(stop words(q1,q2))

ctc_min: This is the ratio of the number of common tokens to the smaller token count among the two questions

eg:#common words tokens/min(tokens(q1,q2))

ctc_max: This is the ratio of the number of common tokens to the larger token count among the two questions

eg:#common words tokens/max(tokens(q1,q2))


last_word_eq: 1 if the last word in the two questions is same, 0 otherwise  

eg:last wale dono match karna chahiye
hello are you
who is you
first_word_eq: 1 if the first word in the two questions is same, 0 otherwise
eg:first wale dono match karna chahiye
hello are you
hello is you

2. Length Based Features
mean_len: Mean of the length of the two questions (number of words)

Eg:using tokens q1 & q2 /2



abs_len_diff: Absolute difference between the length of the two questions (number of words)

Eg:using tokens q1 & q2.q1-q2

longest_substr_ratio: Ratio of the length of the longest substring among the two questions to the length of the smaller question


eg:maximum length of words /minimum length of words


3. Fuzzy Features


fuzz_ratio: fuzz_ratio score from fuzzywuzzy

fuzz_partial_ratio: fuzz_partial_ratio from fuzzywuzzy

token_sort_ratio: token_sort_ratio from fuzzywuzzy

token_set_ratio: token_set_ratio from fuzzywuzzy




In [ ]:
#tolens freatures= tokens=[words],words=stopwords ko chodke all words,stopwords=is,a,the

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('train.csv')


In [3]:
new_df=df.sample(30000,random_state=2)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
204522,204522,307430,307431,What are some good books/papers on generalized...,What is a generalized linear model in layman's...,0.0
61162,61162,106838,106839,What are the things to keep in mind if you wan...,When was the last time you wrote a personal le...,0.0
168520,168520,261015,203759,What should I do if tea tree oil comes in cont...,Does tea tree oil really prevent acne?,0.0
190003,190003,288948,288949,How QiK Stay is diffrent from other Budget hot...,Which is a better budget hotel chain: OYO Room...,0.0
40618,40618,73456,73457,Is it easy to learn Kashmiri language for some...,Why is it so difficult for adult English speak...,0.0


In [4]:
def preprocess(q):
    q=str(q).lower().strip()#lower&remove white spaces
    #replace certain special characters with their string equivalents
    q=q.replace('%', ' percent')
    q=q.replace('$', ' dollar ')
    q=q.replace('₹', ' rupee ')
    q=q.replace('€', ' euro ')
    q=q.replace('@', ' at ')
    #pattern '[math]' appears around 900 times in the whole dataset
    q=q.replace('[math]', '')
    #replacing same numbers with string equvalents
    q=q.replace(',000,000,000 ', 'b ')
    q=q.replace(',000,000 ', 'm ')
    q=q.replace(',000 ', 'k ')
    q=re.sub(r'([0-9]+)000000000', r'\1b', q)
    q=re.sub(r'([0-9]+)000000', r'\1m', q)
    q=re.sub(r'([0-9]+)000', r'\1k', q)

In [5]:
#decontracting words
contractions={
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd":"i would",
    "i'd've":"i would have",
    "i'm":"i am",
     "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }




In [8]:
#
q_decontracted=[]
for word in q.split():
    if word in contractions:
        word=contractions[word]
    q_decontracted.append(word)
    q=' '.join(q_decontracted)
    q=q.replace("'ve", " have")
    q=q.replace("n't", " not")
    q=q.replace("'re", " are")
    q=q.replace("'ll", " will")
    #removing Html tage
    q=BeautifulSoup(q)
    q=q.get_text()
    #remove punctuations
    pattern=re.compile('\W')
    q=re.sub(pattern,' ',q).strip()
  return q


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 18)

In [10]:
def preprocess(q):
    q=str(q).lower().strip()#lower&remove white spaces
    #replace certain special characters with their string equivalents
    q=q.replace('%', ' percent')
    q=q.replace('$', ' dollar ')
    q=q.replace('₹', ' rupee ')
    q=q.replace('€', ' euro ')
    q=q.replace('@', ' at ')
    #pattern '[math]' appears around 900 times in the whole dataset
    q=q.replace('[math]', '')
    #replacing same numbers with string equvalents
    q=q.replace(',000,000,000 ', 'b ')
    q=q.replace(',000,000 ', 'm ')
    q=q.replace(',000 ', 'k ')
    q=re.sub(r'([0-9]+)000000000', r'\1b', q)
    q=re.sub(r'([0-9]+)000000', r'\1m', q)
    q=re.sub(r'([0-9]+)000', r'\1k', q)
    #decontracting words
    q_decontracted=[]
    for word in q.split():
        if word in contractions:
            word=contractions[word]
        q_decontracted.append(word)
        q=' '.join(q_decontracted)
        q=q.replace("'ve", " have")
        q=q.replace("n't", " not")
        q=q.replace("'re", " are")
        q=q.replace("'ll", " will")
        #removing Html tage
        q=BeautifulSoup(q)
        q=q.get_text()
        #remove punctuations
        pattern=re.compile('\W')
        q=re.sub(pattern,' ',q).strip()
    return q #Fixed the indentation of this line

In [11]:
preprocess("I've already! wasn't <b>done</b>?")

'i have already  was not done'

In [12]:
new_df['question1']=new_df['question1'].apply(preprocess)
new_df['question2']=new_df['question2'].apply(preprocess)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
204522,204522,307430,307431,what are some good books papers on generalized...,what is a generalized linear model in layman s...,0.0
61162,61162,106838,106839,what are the things to keep in mind if you wan...,when was the last time you wrote a personal le...,0.0
168520,168520,261015,203759,what should i do if tea tree oil comes in cont...,does tea tree oil really prevent acne,0.0
190003,190003,288948,288949,how qik stay is diffrent from other budget hot...,which is a better budget hotel chain oyo room...,0.0
40618,40618,73456,73457,is it easy to learn kashmiri language for some...,why is it so difficult for adult english speak...,0.0


In [13]:
new_df['q1_len']=new_df['question1'].str.len()
new_df['q2_len']=new_df['question2'].str.len()


In [14]:
new_df['q1_num_words']=new_df['question1'].apply(lambda row:len(row.split(" ")))
new_df['q2_num_words']=new_df['question2'].apply(lambda row:len(row.split(" ")))
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words
204522,204522,307430,307431,what are some good books papers on generalized...,what is a generalized linear model in layman s...,0.0,60,52,10,10
61162,61162,106838,106839,what are the things to keep in mind if you wan...,when was the last time you wrote a personal le...,0.0,126,64,28,13
168520,168520,261015,203759,what should i do if tea tree oil comes in cont...,does tea tree oil really prevent acne,0.0,163,37,37,7
190003,190003,288948,288949,how qik stay is diffrent from other budget hot...,which is a better budget hotel chain oyo room...,0.0,79,68,16,15
40618,40618,73456,73457,is it easy to learn kashmiri language for some...,why is it so difficult for adult english speak...,0.0,148,224,28,42


In [16]:
#for common words
def common_words(row):
    w1=set(map(lambda word:word.lower().strip(),row['question1'].split(" ")))
    w2=set(map(lambda word:word.lower().strip(),row['question2'].split(" ")))
    return len(w1&w2)

In [17]:
new_df['word_common']=new_df.apply(common_words,axis=1)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common
204522,204522,307430,307431,what are some good books papers on generalized...,what is a generalized linear model in layman s...,0.0,60,52,10,10,3
61162,61162,106838,106839,what are the things to keep in mind if you wan...,when was the last time you wrote a personal le...,0.0,126,64,28,13,3
168520,168520,261015,203759,what should i do if tea tree oil comes in cont...,does tea tree oil really prevent acne,0.0,163,37,37,7,3
190003,190003,288948,288949,how qik stay is diffrent from other budget hot...,which is a better budget hotel chain oyo room...,0.0,79,68,16,15,10
40618,40618,73456,73457,is it easy to learn kashmiri language for some...,why is it so difficult for adult english speak...,0.0,148,224,28,42,11


In [18]:
#for total words
def total_words(row):
    w1=set(map(lambda word:word.lower().strip(),row['question1'].split(" ")))
    w2=set(map(lambda word:word.lower().strip(),row['question2'].split(" ")))
    return (len(w1)+len(w2))

In [19]:
new_df['word_total']=new_df.apply(total_words,axis=1)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total
204522,204522,307430,307431,what are some good books papers on generalized...,what is a generalized linear model in layman s...,0.0,60,52,10,10,3,20
61162,61162,106838,106839,what are the things to keep in mind if you wan...,when was the last time you wrote a personal le...,0.0,126,64,28,13,3,36
168520,168520,261015,203759,what should i do if tea tree oil comes in cont...,does tea tree oil really prevent acne,0.0,163,37,37,7,3,40
190003,190003,288948,288949,how qik stay is diffrent from other budget hot...,which is a better budget hotel chain oyo room...,0.0,79,68,16,15,10,29
40618,40618,73456,73457,is it easy to learn kashmiri language for some...,why is it so difficult for adult english speak...,0.0,148,224,28,42,11,59


In [20]:
new_df['word_share']=round(new_df['word_common']/new_df['word_total'],2)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share
204522,204522,307430,307431,what are some good books papers on generalized...,what is a generalized linear model in layman s...,0.0,60,52,10,10,3,20,0.15
61162,61162,106838,106839,what are the things to keep in mind if you wan...,when was the last time you wrote a personal le...,0.0,126,64,28,13,3,36,0.08
168520,168520,261015,203759,what should i do if tea tree oil comes in cont...,does tea tree oil really prevent acne,0.0,163,37,37,7,3,40,0.08
190003,190003,288948,288949,how qik stay is diffrent from other budget hot...,which is a better budget hotel chain oyo room...,0.0,79,68,16,15,10,29,0.34
40618,40618,73456,73457,is it easy to learn kashmiri language for some...,why is it so difficult for adult english speak...,0.0,148,224,28,42,11,59,0.19


In [22]:
#advanced features
from nltk.corpus import stopwords
def fetch_token_features(row):
    q1=row['question1']
    q2=row['question2']
    SAFE_DIV=0.0001
    #stopwords
    stop_words=stopwords.words('english')
    #tokenization
    token_features=[0.0]*8
    #converting string to list
    q1_tokens=q1.split()
    q2_tokens=q2.split()
    if len(q1_tokens)==0 or len(q2_tokens)==0:
        return token_features
    #get the non-stopwords in questions
    q1_words=set([word for word in q1_tokens if word not in stop_words])
    q2_words=set([word for word in q2_tokens if word not in stop_words])
